In [2]:
import numpy as np
import torch
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)
from model import AttentionModel
from dcascore import *
from utils import read_fasta_alignment, remove_duplicate_sequences, add_PCA_coords

# back to original path (in PLM)
sys.path.pop(0)  # Removes the parent_dir from sys.path
from model import AttentionModel

from plm_gen_methods import generate_plm_n_save
from seq_utils import read_tensor_from_txt, set_seed, letters_to_nums, modify_seq 


In [3]:
print(current_dir)
print(parent_dir)

c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src


In [4]:
"""
    Load Q, K, V matrices from jdoms (after training)
"""
set_seed()
H = 64
d= 10
N = 174
n_epochs = 500
loss_type = 'without_J'
family = 'jdoms' #'jdoms_bacteria_train2'
cwd = parent_dir
Q_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/Q_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
K_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/K_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
V_1 = read_tensor_from_txt( cwd +"/results/{H}_{d}_{family}_{losstype}_{n_epochs}_PCA_brute_force_35_bins/V_tensor.txt".format(H=H, d=d, family=family, losstype=loss_type, n_epochs=n_epochs))
H,d,N=Q_1.shape
q=V_1.shape[1]


In [5]:
model=AttentionModel(H,d,N,q,Q=Q_1,V=V_1,K=K_1)
torch.sum(model.Q-Q_1)
device = Q_1.device
L = Q_1.shape[-1]
W=attention_heads_from_model(model,Q_1,K_1,V_1)
print(W.shape)

i_indices = torch.arange(L, device=device).unsqueeze(1)
j_indices = torch.arange(L, device=device).unsqueeze(0)
mask = (i_indices != j_indices).float().unsqueeze(0)  # shape (1, L, L)
W = W * mask
    
# Compute Jtens
Jtens = torch.einsum('hri,hab->abri', W, V_1)  # Shape: (q, q, L, L)
q = Jtens.shape[0]
N = Jtens.shape[2]
print(q)
print(N)

torch.Size([64, 65, 65])
34
65


In [7]:
"""
    Generate sequences with PLM random initialization
"""
save_dir = "generated_sequences"
N_seqs = 30000
save_name = "generated_sequences_randinit_30000"
#generate_plm_n_save(save_dir, save_name, Jtens, N_seqs=N_seqs, init_sequence=None,nb_PCA_comp=2,PCA_comp_list=np.array([20,20]))

##############################################################


In [9]:
"""
    Generate sequences with PLM initialization from a sequence
"""
file_test_data=r"C:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\git_test\PLM-gen-DCA\Attention-DCA-main\CODE\DataAttentionDCA\jdoms\jdoms_bacteria_train2.fasta"
seq_data_test=read_fasta_alignment(file_test_data,0.8)

print(seq_data_test.shape)
seq_data_test_filtered,_=remove_duplicate_sequences(seq_data_test)
seq_data_test_filtered=seq_data_test_filtered.T
seq_data_test_filtered_with_PCA= add_PCA_coords(seq_data_test_filtered,35) #####DONT FORGET TO CHANGE NUMBER OF BINS IF NECESSARY


#init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
init_sequence_num = seq_data_test_filtered_with_PCA[0]
ratio = 0.4
init_sequence_num = modify_seq(init_sequence_num, ratio,nb_PCA_comp=2)
N_seqs=30000
save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}"

generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num)


Total sequences read: 14502
Sequences after filtering: 14502
(63, 14502)
Removing duplicate sequences...
Done: 14502 -> 14502 sequences after removing duplicates.


100%|██████████| 30000/30000 [09:26<00:00, 52.99it/s]


Generated sequences (letters): ['NPYEVLGIKEPWSYDEIKVLARELMKAYHLDQYGDNPLICEKMFVLNEAYDYLSKNS-QF---VG', 'NPYEVLGIKEPWSYDEIKVLARELMKAYHLDQYGDNPLICEKMFELNEAYDYLSKNS-QF---VG', 'NPYEVLGIKEPWSYDENKVLARELMKAYHLDQYGDNPLICEKMFELNEAYDYLSKNS-QF---VG', 'NPYEVLGIKEPKSYDENKVLARELMKAYHLDQYGDNPLICEKMFELNEAYDYLSKNS-QF---VG', 'NPYEVLGIKEPKSYDENKVLARELMKAYHLDQYGDNPLICEKMFELNEAYDYLRKNS-QF---VG']
Generated sequences saved to generated_sequences


In [10]:
"""
    Generate sequences with PLM initialization from a sequence different betas
"""
# init_sequence = 'DYYQVLGVPKDADAKSIKKAFRKLARKYHPDVNPGDKEAERKFKEANEANEVLSDPEKRKKYD'
# init_sequence_num = letters_to_nums(init_sequence)
# ratio = 0.1
# init_sequence_num = modify_seq(init_sequence_num, ratio)

N_seqs=15000
betas = [0.01, 0.1, 0.5, 1, 2, 4, 10]
for b in betas:
    save_name = f"gen_seqs_w_init_seq_Ns{N_seqs}_r{ratio}_b{b}"

    generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b)
    #generate_plm_n_save(save_dir, save_name, Jtens, N_seqs, init_sequence=init_sequence_num, beta=b)

100%|██████████| 15000/15000 [04:47<00:00, 52.09it/s]


Generated sequences (letters): ['-LYEILGVKVDAGVE-IRQAYRRQARRWHPDMH-RNPEAEAAIKEVQRAYEILSNERARGYYDIK', 'MLYEILGVKVDAGVE-IRQAYRRQARRWHPDMH-RNPEAEAAIKEVQRAYEILSNERARGYYDIK', 'MLYEILGVKVDAGVE-IRQAYRRQARRWHPDMH-RNPEAEAAIKEVQRAYEILSNQRARGYYDIK', 'MLYEILGVKVDAGVE-IRQAYRRCARRWHPDMH-RNPEAEAAIKEVQRAYEILSNQRARGYYDIK', 'MLYEILGVCVDAGVE-IRQAYRRCARRWHPDMH-RNPEAEAAIKEVQRAYEILSNQRARGYYDIK']
Generated sequences saved to generated_sequences


100%|██████████| 15000/15000 [04:43<00:00, 52.85it/s]


Generated sequences (letters): ['GKRH-TCDVKIVWWMICMGIWEGPM-DAWFVGNKNRIKFLQYYAVSFEAPGQDWRVRVWYIEYDP', 'GKRH-TCDVKIVWWMICMGIWEGPM-DAWFVGNKNRIKFLQYYAVSFEAPGQDWRVRVWYIEYDP', 'GKRS-TCDVKIVWWMICMGIWEGPM-DAWFVGNKNRIKFLQYYAVSFEAPGQDWRVRVWYIEYDP', 'GKRS-TCDVKIVWWMICMGIWEGPM-DAWFVGNKNRIKFLQYYAVSPEAPGQDWRVRVWYIEYDP', 'GKRS-TCDVKIVWWMICMGIWEGPM-EAWFVGNKNRIKFLQYYAVSPEAPGQDWRVRVWYIEYDP']
Generated sequences saved to generated_sequences


100%|██████████| 15000/15000 [04:46<00:00, 52.40it/s]


...

Generated sequences (letters): ['---------FQSEKAEIQKDYWLSAQRSHPDLCNSEKRMEVIFRQNSQAQDFL-KMFCNS---VC', '---------FQSEKAEIQKDYWLSAQRSHPDLCNSEKRMEVIFRQNSQAQDFL-KMFCNS---VC', '---------FQSEKAEIQKDYWLSAQLSHPDLCNSEKRMEVIFRQNSQAQDFL-KMFCNS---VC', '---------FQSEKAEIQKDYWLSAQLSHPDLCNSEKRMEVIFRANSQAQDFL-KMFCNS---VC', '---------FQSEKAEIQKDYWLSAQLSHPDLCNSEKRMEVIFRANQQAQDFL-KMFCNS---VC']
Generated sequences saved to generated_sequences


100%|██████████| 15000/15000 [04:41<00:00, 53.28it/s]


Generated sequences (letters): ['-AYEVLGIDSDASDEEIKKAYRRLALKYHPDKG----GSAEKFREVQEAYEAIK----------H', '-AYEVLGIDSNASDEEIKKAYRRLALKYHPDKG----GSAEKFREVQEAYEAIK----------H', '-AYEVLGIDSNASDEEIKKAYRRLALKYHPDKG----GSAEKFREVQEAYEVIK----------H', '-AYEVLGIDSNASDEEIKKAYRRLALKYHPDKG----GSAEKFREVQEAYEVIK----------H', '-AYEVLGIDSNASDEEIKKAYRRLALKYHPDKG----GSAEKFREVQEAYEVIK----------H']
Generated sequences saved to generated_sequences


  0%|          | 12/15000 [00:00<04:43, 52.83it/s]c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:52: RuntimeWarning: overflow encountered in exp
  prob = np.exp(self.beta * sum_energy)  # unnormalized
c:\Users\youss\OneDrive\Bureau\master epfl\MA2\TP4 De los Rios\PCA_gen_ai\PCA_gen_AI\CODE\AttentionDCA_python\src\PLM\plm_model.py:63: RuntimeWarning: invalid value encountered in divide
  probs /= probs.sum()
  0%|          | 15/15000 [00:00<05:00, 49.81it/s]


ValueError: probabilities contain NaN